In [76]:
import numpy as np
import pandas as pd
import scipy

In [117]:
X = np.random.rand(100, 10)

dist_matrix = scipy.spatial.distance_matrix(X, X)
pd.DataFrame(dist_matrix)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,1.225407,1.779884,1.444598,1.512182,1.387179,1.295974,1.772115,1.437804,0.991773,...,1.190618,1.130912,1.128513,1.563696,1.171768,1.746869,1.616962,1.462317,1.562953,1.503998
1,1.225407,0.000000,1.465953,1.025015,1.215155,1.010927,1.232184,1.409755,1.219533,1.146057,...,0.797767,1.045253,1.170697,1.170461,0.854590,1.393549,1.177159,1.291754,1.373322,1.422567
2,1.779884,1.465953,0.000000,1.557517,1.436343,1.586270,1.234891,1.346089,1.333969,1.182426,...,1.325120,1.473191,1.495732,1.284530,1.533119,1.233982,1.147197,1.430252,1.542606,1.737807
3,1.444598,1.025015,1.557517,0.000000,1.128518,0.931045,1.139882,1.062616,0.956373,1.294366,...,1.179209,1.287747,0.924003,1.233599,0.952372,1.163743,1.046409,1.199185,1.053483,1.204507
4,1.512182,1.215155,1.436343,1.128518,0.000000,1.560575,1.719199,1.536356,1.193603,0.816761,...,1.563281,1.189640,1.239406,1.363473,1.398866,1.119708,1.102032,1.612802,1.024919,1.215219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.746869,1.393549,1.233982,1.163743,1.119708,1.272473,1.341658,0.836324,1.531092,1.211447,...,1.471115,1.191460,1.112487,0.864609,1.189554,0.000000,1.311269,1.167017,1.016944,1.222189
96,1.616962,1.177159,1.147197,1.046409,1.102032,1.309466,1.403584,1.594401,0.861834,1.100084,...,1.311754,1.604122,1.524112,1.390595,1.442374,1.311269,0.000000,1.213934,1.302735,1.505824
97,1.462317,1.291754,1.430252,1.199185,1.612802,0.729542,1.108483,1.234443,1.394201,1.357059,...,1.100930,1.492705,1.356055,0.795275,1.061579,1.167017,1.213934,0.000000,1.034886,1.231527
98,1.562953,1.373322,1.542606,1.053483,1.024919,1.163025,1.581017,1.205318,1.380834,1.189596,...,1.349505,1.308145,1.238895,0.987355,1.284405,1.016944,1.302735,1.034886,0.000000,1.012384


In [245]:
def reachability_distance(k, dist_matrix):
    k_neighbors_index = np.argsort(dist_matrix, axis=1)[:, 1:k+1]
    reachability_dist = np.zeros((len(k_neighbors_index), k))
    for i in range(len(k_neighbors_index)):
        # 点iの、近傍k個の点との距離
        a = dist_matrix[i, k_neighbors_index[i]]
        # 点iの、近傍k個の点それぞれにおける、さらに近傍k個の点との距離
        b = dist_matrix[k_neighbors_index[i], k_neighbors_index[k_neighbors_index[i]]]
        reachability_dist[i] = np.maximum(a, b.max(axis=1))

    return reachability_dist, k_neighbors_index

def local_reachability_density(reachability_dist):
    return (1. / np.mean(reachability_dist, axis=1))

def local_outlier_factor(lrd, k_neibors_index):
    return (lrd[k_neibors_index].mean(axis=1) / lrd) / len(lrd)

In [244]:
reachability_dist, k_neibors_index = reachability_distance(3, dist_matrix)
lrd = local_reachability_density(reachability_dist)
lrd[k_neibors_index].mean(axis=1)

array([0.88244024, 0.91433688, 0.78853706, 0.93153187, 0.95712272,
       0.94293942, 0.99058589, 0.85961199, 1.10821497, 0.87632813,
       0.84260614, 0.96126208, 0.95104265, 0.84076428, 0.82391792,
       1.20490256, 0.9677001 , 0.81464357, 0.95080448, 1.01782642,
       0.98407664, 1.13362392, 0.97438791, 0.94980174, 0.96688769,
       0.96768939, 1.05769914, 0.95569399, 0.79349638, 1.06279324,
       0.93407798, 1.21481516, 1.02903888, 1.11857609, 0.9114989 ,
       1.01134103, 0.87052533, 0.89568103, 1.00048986, 0.79812723,
       0.85049267, 0.88598349, 0.93245102, 0.8788645 , 1.04205112,
       0.88198373, 1.02414109, 0.94381369, 0.94637418, 0.96564244,
       1.0707241 , 0.91399975, 0.78109882, 0.97834735, 1.00644083,
       0.95786747, 1.13128352, 0.86112349, 0.8818683 , 0.82008378,
       0.88212852, 0.93946899, 1.02268744, 0.99642692, 1.02723216,
       0.89674792, 0.87648988, 1.07324909, 0.89823819, 1.11428309,
       0.81347819, 0.89479188, 0.88524167, 1.10821497, 0.98097

In [ ]:
# k距離を計算します。
def k_distance(k, distance_matrix):
    # 各行をソートします（距離行列内の各点から他の全ての点までの距離）。
    sorted_distances = np.sort(distance_matrix, axis=1)
    # k番目の距離を返します。
    return sorted_distances[:, k]

# 到達距離を計算します。
def reachability_distance(k, distance_matrix):
    # k距離を取得します。
    k_distances = k_distance(k, distance_matrix)
    # 各距離とk距離の大きい方を取ります。
    reachability_distance_matrix = np.maximum(distance_matrix, k_distances)
    return reachability_distance_matrix

# 局所到達可能密度（LRD）を計算します。
def local_reachability_density(k, distance_matrix):
    # 到達距離を計算します。
    reachability_distance_matrix = reachability_distance(k, distance_matrix)
    # LRDは、k近傍点までの平均到達距離の逆数として定義されます。
    return 1. / (np.sum(reachability_distance_matrix, axis=1) / k)

# Local Outlier Factor（LOF）を計算します。
def local_outlier_factor(k, distance_matrix):
    # 局所到達可能密度を計算します。
    lrd = local_reachability_density(k, distance_matrix)
    # 比率行列を計算します（各点のLRDをそのk近傍点のLRDで割ります）。
    ratio_matrix = lrd / lrd[:, None]
    # LOFは、比率の平均として定義されます。
    return np.sum(ratio_matrix, axis=1) / k
